In [14]:
#r "../.nuget/packages/fsharp.systemtextjson/0.14.8/lib/netstandard2.0/FSharp.SystemTextJson.dll"

In [15]:
#r "../src/bin/Debug/net5.0/HafasJsonRpcClient.dll"

In [16]:
open HafasLibrary

In [17]:
open Html

Start the node process.

In [18]:
let client = startClient (HafasLibrary.Profile.Db, { defaultClientOptions with
               script = "../../hafas-jsonrpc-server/build/index.js" })

Load the uic identifiers.

In [19]:
let maybeFrom = getIdOfFirstStop (getLocations client "Köln" None)

In [20]:
let maybeTo = getIdOfFirstStop (getLocations client "Berlin" None)

Load the journeys.

In [21]:
let journeysOptions =
    { defaultJourneysOptions with
          results = Some 5
          stopovers = Some true}

In [22]:
let maybeJourneys =
    match maybeFrom, maybeTo with
    | Some from, Some ``to`` ->
        match (getJourneys client from ``to`` (Some journeysOptions)) with
        | Some result -> result.journeys
        | _ -> None
    | _ -> None

Extract journey summaries.

In [23]:
match maybeJourneys with
| Some journeys ->
    journeys
    |> Array.map getJourneySummary
    |> Array.choose id
| _ -> Array.empty

index,origin,departure,destination,arrival
0,Köln Hbf,2020-09-06T17:48:00+02:00,Berlin Hbf (tief),2020-09-06T22:18:00+02:00
1,Köln Hbf,2020-09-06T18:48:00+02:00,Berlin Hbf (tief),2020-09-06T23:05:00+02:00
2,Köln Hbf,2020-09-06T19:48:00+02:00,Berlin Hbf (tief),2020-09-07T00:10:00+02:00
3,Köln Hbf,2020-09-06T20:48:00+02:00,Berlin Hbf (tief),2020-09-07T01:10:00+02:00
4,Köln Hbf,2020-09-06T22:39:00+02:00,Berlin Hbf,2020-09-07T05:36:00+02:00


In [24]:
let maybeFirstJourney =
    match maybeJourneys with
    | Some journeys -> if journeys.Length > 0 then Some journeys.[0] else None
    | _ -> None

In [25]:
let journeyLocations =
    match maybeFirstJourney with
    | Some journey -> getJourneyLocations journey
    | None -> Array.empty

In [26]:
display(iframe [_src (BRouter.getUri journeyLocations); _width "100%"; _height "450"] [])